In [1]:
import time 
start_alltime = time.time()## точка отсчета времени
import cv2
import numpy as np
import os as os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.metrics import *
from sklearn import preprocessing
from scipy import signal as sg
from sklearn.neighbors import KNeighborsClassifier

In [2]:
def laws_texture(gray_image):
    (rows, cols) = gray_image.shape[:2]
    smooth_kernel = (1/25)*np.ones((5,5))
    gray_smooth = sg.convolve(gray_image, smooth_kernel, 'same')
    gray_processed = np.abs(gray_image - gray_smooth)
    
    filter_vectors = np.array([[1,4,6,4,1],
                              [-1,-2,0,2,1],
                              [-1,0,2,0,1],
                              [1,-4,6,-4,1]])
    filters = []
    for i in range(4):
        for j in range(4):
            filters.append(np.matmul(filter_vectors[i][:].reshape(5,1),
                               filter_vectors[j][:].reshape(1,5)))
    
    conv_maps = np.zeros((rows, cols, 16))
    for i in range(len(filters)):
        conv_maps[:,:,i] = sg.convolve(gray_processed,
                                      filters[i], 'same')
    
    texture_maps = list()
    texture_maps.append((conv_maps[:,:,1]+conv_maps[:,:,4])//2)
    texture_maps.append((conv_maps[:,:,2]+conv_maps[:,:,8])//2)
    texture_maps.append((conv_maps[:,:,3]+conv_maps[:,:,12])//2)
    texture_maps.append((conv_maps[:,:,7]+conv_maps[:,:,13])//2)
    texture_maps.append((conv_maps[:,:,6]+conv_maps[:,:,9])//2)
    texture_maps.append((conv_maps[:,:,11]+conv_maps[:,:,14])//2)
    texture_maps.append((conv_maps[:,:,10]))
    texture_maps.append((conv_maps[:,:,5]))
    texture_maps.append((conv_maps[:,:,15]))
    texture_maps.append((conv_maps[:,:,0])) 
    
    TEM = list()
    for i in range(9):
        TEM.append(np.abs(texture_maps[i]).sum() /
                  np.abs(texture_maps[9]).sum())
    return TEM

## Load and preprocessing image ##

In [3]:
def LaP_images(path):
    images = []
    filenames = os.listdir(path)
    for filename in filenames:    
        image = cv2.imread(os.path.join(path, filename),0)
        image = cv2.resize(image, dsize=(150,150))
        images.append(image)
    return np.array(images)

In [4]:
start_LaP_time = time.time()## точка отсчета времени
foil_train_dir = os.getcwd() + "\\Desert"
corduroy_train_dir = os.getcwd() + "\\DenseResidential"
cork_train_dir = os.getcwd() + "\\Forest"
catton_train_dir = os.getcwd() + "\\Meadow"
linen_train_dir = os.getcwd() + "\\River"

In [5]:
foil_train_list = LaP_images(foil_train_dir)
corduroy_train_list = LaP_images(corduroy_train_dir)
cork_train_list = LaP_images(cork_train_dir)
catton_train_list = LaP_images(catton_train_dir)
linen_train_list = LaP_images(linen_train_dir)

In [6]:
print(len(foil_train_list[1]))
print(len(foil_train_list))
print(len(corduroy_train_list))
print(len(cork_train_list))
print(len(catton_train_list))
print(len(linen_train_list))

150
217
217
217
217
217


### All image in one array ###

In [7]:
start_EX_time = time.time()## точка отсчета времени
all_images = np.vstack((foil_train_list,
                        corduroy_train_list,
                        cork_train_list,
                        catton_train_list,
                        linen_train_list
                        ))
print(all_images.shape,"Shape before extract")

(1085, 150, 150) Shape before extract


In [8]:
labels = []
for i in range(len(foil_train_list)):
    labels.append(0)
for i in range(len(corduroy_train_list)):
    labels.append(1)
for i in range(len(cork_train_list)):
    labels.append(2)
for i in range(len(catton_train_list)):
    labels.append(3)
for i in range(len(linen_train_list)):
    labels.append(4)
    
labels = np.array(labels)
print("--- %s seconds Load and preprocessing ---" % (time.time() - start_LaP_time)) ## вывод время работы программы

--- 3.2169222831726074 seconds Load and preprocessing ---


## Extract Laws Texture Energy Measures ##

In [9]:
def extract_laws(images):
    tems = []
    for image in images:
        law = laws_texture(image)
        tems.append(law)
    
    return np.array(tems)

In [10]:
start_EX_time = time.time()## точка отсчета времени
X_all_laws = extract_laws(all_images)
print("--- %s seconds Extract Laws Features ---" % (time.time() - start_EX_time)) ## вывод время работы программы

--- 19.34657573699951 seconds Extract Laws Features ---


In [11]:
print(X_all_laws.shape,"Shape after extract")

(1085, 9) Shape after extract


## General dataset ##

In [12]:
col = ["L5E5 / E5L5", "L5S5 / S5L5", "L5R5 / R5L5", "E5R5 / R5E5", "E5S5 / S5E5", "S5R5 / R5S5", "S5S5", "E5E5", "R5R5"]

In [13]:
result = pd.DataFrame(data = X_all_laws,columns = col)

In [14]:
result

,L5E5 / E5L5,L5S5 / S5L5,L5R5 / R5L5,E5R5 / R5E5,E5S5 / S5E5,S5R5 / R5S5,S5S5,E5E5,R5R5
0,0.090781,0.139365,0.140277,0.028190,0.015869,0.031138,0.023207,0.017176,0.096574
1,0.058706,0.128299,0.122154,0.039761,0.014044,0.036813,0.019910,0.022993,0.147837
2,0.103672,0.149097,0.145773,0.018413,0.016026,0.027059,0.025044,0.011804,0.064441
3,0.086811,0.136177,0.134686,0.032209,0.016287,0.032480,0.022600,0.020899,0.108868
4,0.102617,0.146685,0.143259,0.020331,0.016172,0.026896,0.024512,0.013717,0.067039
...,...,...,...,...,...,...,...,...,...
1080,0.067842,0.129439,0.117752,0.037825,0.015199,0.034482,0.020323,0.024252,0.132877
1081,0.065831,0.128917,0.117716,0.035902,0.014513,0.033454,0.019973,0.022872,0.126178
1082,0.069880,0.128726,0.116989,0.036997,0.015170,0.033184,0.020202,0.023717,0.122938
1083,0.069178,0.130316,0.123452,0.038246,0.015201,0.034631,0.020670,0.023596,0.127838


## Classification ##

In [15]:
X = result

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

Y = labels

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size = 0.30)

In [16]:
start_SVM_time = time.time()## точка отсчета времени
clf_svm = svm.SVC()
param_grid = {'C':[50,100,150],'gamma':[0.001,0.01,0.1]}#param_grid = {'C':[5,50,100,150,200,250,300,350,400],'gamma':[0.01,0.1,0.5]}
grid_search_svm_clf=GridSearchCV(clf_svm,param_grid, refit = True, verbose=1)
grid_search_svm_clf.fit(X_train, Y_train)
print("--- %s seconds SVM+GridSearch fit ---" % (time.time() - start_SVM_time)) ## вывод время работы программы

Fitting 5 folds for each of 9 candidates, totalling 45 fits
--- 0.3683350086212158 seconds SVM+GridSearch fit ---


In [17]:
grid_search_svm_clf.best_estimator_

SVC(C=100, gamma=0.01)

In [18]:
start_PR_time = time.time()## точка отсчета времени
best_svm = grid_search_svm_clf.best_estimator_
Y_pred_svm = best_svm.predict(X_test)
print("Accuracy:",accuracy_score(Y_test, Y_pred_svm))
print(classification_report(Y_test, Y_pred_svm))
print("--- %s seconds Predict time ---" % (time.time() - start_PR_time)) ## вывод время работы программы

Accuracy: 0.8834355828220859
              precision    recall  f1-score   support

           0       0.93      0.86      0.90        66
           1       0.96      0.91      0.93        75
           2       0.89      0.96      0.92        67
           3       0.85      0.86      0.85        58
           4       0.78      0.82      0.80        60

    accuracy                           0.88       326
   macro avg       0.88      0.88      0.88       326
weighted avg       0.89      0.88      0.88       326

--- 0.009008407592773438 seconds Predict time ---


In [19]:
print("--- %s seconds ALL  ---" % (time.time() - start_alltime)) ## вывод время работы программы

--- 23.786608457565308 seconds ALL  ---


In [20]:
start_KNN_time = time.time()## точка отсчета времени

model_knn = KNeighborsClassifier(n_neighbors=1)
model_knn.fit(X_train, Y_train)

print('KNN train acc\t:', model_knn.score(X_train, Y_train))
print('KNN test acc\t:', model_knn.score(X_test, Y_test))

print("--- %s seconds KNN ---" % (time.time() - start_KNN_time)) ## вывод время работы программы

KNN train acc	: 1.0
KNN test acc	: 0.8680981595092024
--- 0.023021221160888672 seconds KNN ---


In [21]:
start_TEST_time = time.time()## точка отсчета времени

foil_test_dir = os.getcwd() + "\\Desert_test"
corduroy_test_dir = os.getcwd() + "\\DenseResidential_test"
cork_test_dir = os.getcwd() + "\\Forest_test"
catton_test_dir = os.getcwd() + "\\Meadow_test"
linen_test_dir = os.getcwd() + "\\River_test"


foil_test_list = LaP_images(foil_test_dir)
corduroy_test_list = LaP_images(corduroy_test_dir)
cork_test_list = LaP_images(cork_test_dir)
catton_test_list = LaP_images(catton_test_dir)
linen_test_list = LaP_images(linen_test_dir)


all_test_images = np.vstack((foil_test_list,
                        corduroy_test_list,
                        cork_test_list,
                        catton_test_list,
                        linen_test_list
                        ))

labels_test = []
for i in range(len(foil_test_list)):
    labels_test.append(0)
for i in range(len(corduroy_test_list)):
    labels_test.append(1)
for i in range(len(cork_test_list)):
    labels_test.append(2)
for i in range(len(catton_test_list)):
    labels_test.append(3)
for i in range(len(linen_test_list)):
    labels_test.append(4)

    
labels_test = np.array(labels_test)

In [22]:
X_test_laws = extract_laws(all_test_images)
result_test = pd.DataFrame(data = X_test_laws,columns = col)

In [23]:
result_test

,L5E5 / E5L5,L5S5 / S5L5,L5R5 / R5L5,E5R5 / R5E5,E5S5 / S5E5,S5R5 / R5S5,S5S5,E5E5,R5R5
0,0.097412,0.144083,0.141584,0.021937,0.015592,0.028200,0.023953,0.012630,0.077383
1,0.086099,0.136376,0.130626,0.029783,0.016041,0.030383,0.022484,0.020421,0.096229
2,0.071913,0.132580,0.121460,0.031617,0.014185,0.031297,0.020903,0.018941,0.110749
3,0.076667,0.131554,0.127051,0.033407,0.015056,0.030630,0.020893,0.021179,0.099565
4,0.096668,0.139320,0.142549,0.034407,0.018496,0.034686,0.024150,0.023315,0.115426
...,...,...,...,...,...,...,...,...,...
160,0.078119,0.130910,0.123912,0.042818,0.017519,0.038868,0.021649,0.027794,0.153621
161,0.070377,0.129148,0.119309,0.041693,0.017110,0.036998,0.021349,0.028702,0.144894
162,0.059535,0.127165,0.124704,0.043745,0.014981,0.039569,0.020185,0.024744,0.162900
163,0.055778,0.127400,0.126305,0.043308,0.014114,0.039081,0.019874,0.023378,0.156774


In [24]:
start_TEPR_time = time.time()## точка отсчета времени
Xtest = result_test

scaler = preprocessing.StandardScaler().fit(Xtest)
X_sc_test = scaler.transform(Xtest)

Ytest = labels_test

In [25]:
Y_pred_test = best_svm.predict(X_sc_test)

In [26]:
print(classification_report(Ytest, Y_pred_test))

              precision    recall  f1-score   support

           0       0.93      0.85      0.89        33
           1       0.94      1.00      0.97        33
           2       0.97      0.94      0.95        33
           3       0.78      0.85      0.81        33
           4       0.78      0.76      0.77        33

    accuracy                           0.88       165
   macro avg       0.88      0.88      0.88       165
weighted avg       0.88      0.88      0.88       165



In [27]:
print("--- %s seconds Test_predict ---" % (time.time() - start_TEPR_time))
print("--- %s seconds test ---" % (time.time() - start_TEST_time))

--- 0.04804372787475586 seconds Test_predict ---
--- 3.515193462371826 seconds test ---
